In [1]:
import stf

In [2]:
from pathlib import Path

verbose= False

config_path = './release_211223/front_config.json'
reference_face = './release_211223/hunet.png'

template_video_path = './release_211223/demo_src/Est Soft 2_1.0-80.mov' # 보라색 자켓, 고개 끄덕 템플릿 (고개 간혹 끄덕임 주문했으나 거의 정자세)
work_root_path = './stf_api_temp_root'
device = 'cuda:1'
checkpoint_path = f'./release_211223/0119_180.pth'



In [3]:
%%time 
def process_callback(per: int) -> None:
    print(per)
    
# template video 전처리
stf.preprocess_template(config_path, template_video_path,
                        reference_face=reference_face,
                        work_root_path=work_root_path,
                        call_back=process_callback,
                        device=device,
                        verbose=verbose)

load MTCNN  success ^ ^
load g_recognizer  success ^ ^
save_face_info2 - df_face_info:  stf_api_temp_root/preprocess/hunet_v1_front/df_face_info/Est Soft 2_1.0-80.pickle


extract frame stride(30) Est Soft 2_1.0-80.mov:   0%|          | 0/80 [00:00<?, ?it/s]

find_faces for calc_ebd:   0%|          | 0/80 [00:00<?, ?it/s]

df_anchor_i: stf_api_temp_root/preprocess/hunet_v1_front/df_anchor_i/Est Soft 2_1.0-80_000.pickle
mean_box: [1638, 205, 2085, 748]   width: 448  height: 544


extract frame stride(1) Est Soft 2_1.0-80.mov:   0%|          | 0/2400 [00:00<?, ?it/s]

■ fan :   0%|          | 0/2400 [00:00<?, ?it/s]

CPU times: user 11min 25s, sys: 1min 40s, total: 13min 6s
Wall time: 4min 56s


In [4]:
%%time 
model = stf.create_model(config_path, checkpoint_path, work_root_path, device=device, verbose=verbose)

CPU times: user 4.76 s, sys: 1.15 s, total: 5.92 s
Wall time: 3.49 s


In [5]:
%%time 
template = stf.template_video(model, template_video_path, callback=None, verbose=verbose)

CPU times: user 18.4 s, sys: 53.5 s, total: 1min 11s
Wall time: 1min 40s


In [6]:
%%time 
from pathlib import Path

for i in range(1):
    print('###################### ', i, ' ##############################')
    wav_path = 'release_211223/demo_src/hunet_only_voice_test_50000.wav'
    wav_std=True
    wav_std_ref_wav=None
    name_prefix=''
    slow_write = True # False
    video_start_offset_frame = 0 # 0이상 숫자: video 해당 frame부터 합성 / None: 맨 마지막에 맞춰서 합성
    
    out_path = 'temp_result/' + f'{name_prefix}_{Path(template_video_path).stem}_{Path(wav_path).stem}_mask-{template.model.args.mask_ver}_{i}.mp4'
    template.gen(wav_path, wav_std, wav_std_ref_wav,
                 video_start_offset_frame, out_path=out_path,
                 slow_write=slow_write)

######################  0  ##############################
mel_step_size:108, mel_ps:80
CPU times: user 1min 16s, sys: 3min 21s, total: 4min 37s
Wall time: 3min 30s


In [7]:
import IPython.display as ipd
ipd.Video(out_path, width=1280)

In [8]:

try:
    print('del template')
    del template
except:
    print('del template exception')
    pass

try:
    print('del model')
    del model
except:
    print('del model exception')
    pass



del template
del model
